In [1]:
import json
from datetime import datetime
from pathlib import Path

import albumentations as A
import cv2
import numpy as np
import polars as pl
import pytz
import timm
import torch
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from omegaconf import OmegaConf
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import get_cosine_schedule_with_warmup

from src.config import cfg
from src.dir import create_dir
from src.seed import seed_everything

cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)
pl.Config.set_fmt_str_lengths(1000)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


/home/marumarukun/pj/compe/atma_18/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


exp_number: '012'
run_time: base
data:
  input_root: ../../data/input
  train_path: ../../data/input/train_features.csv
  test_path: ../../data/input/test_features.csv
  sample_submission_path: ../../data/input/sample_submission.csv
  img_root: ../../data/input/images
  json_root: ../../data/input/traffic_lights
  depth_root: ../../data/input/depth
  output_root: ../../data/output
  results_root: ../../results
  results_path: ../../results/012/base
seed: 319
n_splits: 5
target_cols:
- x_0
- y_0
- z_0
- x_1
- y_1
- z_1
- x_2
- y_2
- z_2
- x_3
- y_3
- z_3
- x_4
- y_4
- z_4
- x_5
- y_5
- z_5
cnn:
  model_name: swin_base_patch4_window7_224.ms_in22k_ft_in1k
  size: 224
  pretrained: true
  in_chans: 20
  target_size: 18
  lr: 0.001
  num_epochs: 6
  batch_size: 32

Using device: cuda


### exp012

- late sub
- 信号機のマスク情報とdepth画像をチャネル方向に結合して入力とするNN
- 数値特徴量も入力とする
- 全結合層を4層に
- targetを標準化して学習する → 推論時にスケールを戻す
- backbone：swin_base_patch4_window12_384.ms_in22k_ft_in1k

### データの読み込み

In [2]:
# データの読み込み
train = pl.read_csv(cfg.data.train_path, try_parse_dates=True).with_row_index()
test = pl.read_csv(cfg.data.test_path, try_parse_dates=True).with_row_index()
sample_submission = pl.read_csv(cfg.data.sample_submission_path, try_parse_dates=True).with_row_index()


# scene列を作成 → これでGroupKFoldする
train = train.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))
test = test.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))

# データの結合(label encoding用)
train_test = pl.concat([train, test], how="diagonal")

# CV
gkf = GroupKFold(n_splits=cfg.n_splits)


In [3]:
# 数値特徴量の前処理
numeric_cols = ["vEgo", "aEgo", "steeringAngleDeg", "steeringTorque", "gas"]

# 欠損を埋める
train_test = train_test.with_columns([pl.col(col).fill_null(0) for col in numeric_cols])

# 全ての数値特徴量を標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(train_test[numeric_cols].to_numpy())

# 標準化した値でDataFrameを更新
train_test = train_test.with_columns(
    [pl.Series(scaled_features[:, i]).alias(col) for i, col in enumerate(numeric_cols)]
)


In [4]:
# train, testに分ける
train = train_test.filter(pl.col("x_0").is_not_null()).sort("index")
test = train_test.filter(pl.col("x_0").is_null()).sort("index")


In [5]:
# targetを標準化
target_scaler = StandardScaler()
scaled_targets = target_scaler.fit_transform(train[cfg.target_cols].to_numpy())  # これは予測値に対しても使う
train_target_scaled = train.with_columns(
    [pl.Series(scaled_targets[:, i]).alias(col) for i, col in enumerate(cfg.target_cols)]
)


### データ拡張


In [6]:
def get_train_transform():
    return A.ReplayCompose(
        [
            A.Resize(cfg.cnn.size, cfg.cnn.size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


def get_valid_transform():
    return A.ReplayCompose(
        [
            A.Resize(cfg.cnn.size, cfg.cnn.size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]
    )


### 信号機マスク

In [7]:
class TrafficLightMaskGenerator:
    TRAFFIC_LIGHT_CLASSES = ["green", "yellow", "red", "straight", "left", "right", "empty", "other"]

    def __init__(self, image_size: list[int, int] | int | None = None):
        self.original_size = (128, 64)  # (width, height)

        if isinstance(image_size, list):
            self.image_size = (image_size[1], image_size[0])
        elif isinstance(image_size, int):
            self.image_size = (image_size, image_size)
        else:
            self.image_size = self.original_size

        self.scale = (self.image_size[0] / self.original_size[0], self.image_size[1] / self.original_size[1])

        self.class_to_index = {cls: idx for idx, cls in enumerate(self.TRAFFIC_LIGHT_CLASSES)}

    def scale_bbox(self, bbox):
        x1, y1, x2, y2 = bbox
        return [x1 * self.scale[0], y1 * self.scale[1], x2 * self.scale[0], y2 * self.scale[1]]

    def _convert_coordinate(self, coord, max_size):
        """座標を適切な範囲に収める"""
        return max(0, min(int(round(coord)), max_size - 1))

    def generate_masks(self, traffic_lights_json):
        # PosixPathオブジェクトを文字列に変換
        if isinstance(traffic_lights_json, Path):
            traffic_lights_json = str(traffic_lights_json)

        traffic_lights = (
            json.load(open(traffic_lights_json)) if isinstance(traffic_lights_json, str) else traffic_lights_json
        )

        masks = np.zeros((self.image_size[1], self.image_size[0], len(self.TRAFFIC_LIGHT_CLASSES)), dtype=np.float32)

        for light in traffic_lights:
            class_idx = self.class_to_index[light["class"]]
            scaled_bbox = self.scale_bbox(light["bbox"])

            # 座標を適切な範囲に収める
            x1 = self._convert_coordinate(scaled_bbox[0], self.image_size[0])
            y1 = self._convert_coordinate(scaled_bbox[1], self.image_size[1])
            x2 = self._convert_coordinate(scaled_bbox[2], self.image_size[0])
            y2 = self._convert_coordinate(scaled_bbox[3], self.image_size[1])

            masks[y1 : y2 + 1, x1 : x2 + 1, class_idx] = 1.0

        return masks


### データセット

In [8]:
class CustomDataset(Dataset):
    def __init__(self, df, img_dir, json_dir, depth_dir, transform=None, is_train=True):
        self.df = df
        self.img_dir = Path(img_dir)
        self.json_dir = Path(json_dir)
        self.depth_dir = Path(depth_dir)
        self.is_train = is_train
        self.feature_cols = [
            "vEgo",
            "aEgo",
            "steeringAngleDeg",
            "steeringTorque",
            "gas",
            "leftBlinker",
            "rightBlinker",
            "brakePressed",
        ]

        # オリジナル画像用の変換処理
        if transform is None:
            self.transform = A.ReplayCompose(
                [
                    A.Resize(cfg.cnn.size, cfg.cnn.size),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406],  # 通常のImageNet平均値
                        std=[0.229, 0.224, 0.225],  # 通常のImageNet標準偏差
                    ),
                    ToTensorV2(),
                ]
            )
        else:
            self.transform = transform

        # Depth Map用の変換処理
        self.depth_transform = A.ReplayCompose(
            [
                A.Resize(cfg.cnn.size, cfg.cnn.size),
                A.Normalize(mean=[0.5], std=[0.5]),
                ToTensorV2(),
            ]
        )

        self.target_cols = cfg.target_cols
        self.mask_generator = TrafficLightMaskGenerator(cfg.cnn.size)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df[idx]
        img_folder = self.img_dir / row["ID"].item()
        depth_folder = self.depth_dir / row["ID"].item()
        json_path = self.json_dir / f"{row['ID'].item()}.json"

        # 3枚の画像を読み込み
        img_names = ["image_t-1.0.png", "image_t-0.5.png", "image_t.png"]
        imgs = []
        depths = []

        for img_name in img_names:
            # オリジナル画像の読み込み
            img_path = img_folder / img_name
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)

            # Depth Mapの読み込み
            depth_path = depth_folder / img_name
            depth = cv2.imread(str(depth_path), cv2.IMREAD_GRAYSCALE)
            depths.append(depth)

        # オリジナル画像の変換
        if self.transform:
            replay = None
            transformed_imgs = []

            for img in imgs:
                if replay is None:
                    # 最初の画像に対して変換を実行し、replayを保存
                    transformed = self.transform(image=img)
                    replay = transformed["replay"]
                else:
                    # 2枚目以降は保存したreplayを使用
                    transformed = A.ReplayCompose.replay(replay, image=img)
                transformed_imgs.append(transformed["image"])

        # Depth Mapの変換
        transformed_depths = []
        for depth in depths:
            transformed_depth = self.depth_transform(image=depth)
            transformed_depths.append(transformed_depth["image"])

        # オリジナル画像をチャネル方向に結合 (C*3, H, W)
        img_tensor = torch.cat(transformed_imgs, dim=0)

        # Depth Mapをチャネル方向に結合 (3, H, W)
        depth_tensor = torch.cat(transformed_depths, dim=0)

        # 信号機マスクの生成
        mask = self.mask_generator.generate_masks(json_path)
        mask_tensor = torch.from_numpy(mask).permute(2, 0, 1)  # (8, H, W)

        # 画像, Depth Map, 信号機マスクをチャネル方向に結合 (C*3+3+8=20, H, W)
        combined_tensor = torch.cat([img_tensor, depth_tensor, mask_tensor], dim=0)

        # 数値特徴量の取得
        feature_tensor = torch.tensor(row[self.feature_cols].to_numpy(), dtype=torch.float32).squeeze(0)

        # ターゲットの準備
        if self.is_train:
            target = torch.tensor(row[self.target_cols].to_numpy(), dtype=torch.float32).squeeze(0)
            return combined_tensor, feature_tensor, target
        else:
            return combined_tensor, feature_tensor


### モデル

In [9]:
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False, target_size=None, model_name=None):
        super().__init__()

        self.encoder = timm.create_model(
            cfg.model_name,
            pretrained=cfg.pretrained,
            num_classes=0,
            in_chans=cfg.in_chans,
        )

        self.n_features = self.encoder.num_features
        self.n_numeric_features = 8
        self.target_size = cfg.target_size if target_size is None else target_size

        # 数値特徴量用のMLP
        self.numeric_mlp = nn.Linear(self.n_numeric_features, 128)

        # 結合後の特徴量の次元
        head_dim = self.n_features + 128

        # ヘッド部分を残差接続を使用する形に変更
        self.head1 = nn.Linear(head_dim, head_dim)
        self.head2 = nn.Linear(head_dim, head_dim)
        self.head3 = nn.Linear(head_dim, head_dim)
        self.final_layer = nn.Linear(head_dim, self.target_size)

    def forward(self, image, numeric_features):
        # 画像特徴量の抽出
        x = self.encoder(image)

        # 数値特徴量の処理
        f = self.numeric_mlp(numeric_features)

        # 特徴量の結合
        x = torch.cat([x, f], dim=-1)
        x = F.gelu(x)

        # 残差接続を使用したヘッド部分
        x_res = F.gelu(self.head1(x))
        x = x + x_res

        x_res = F.gelu(self.head2(x))
        x = x + x_res

        x_res = F.gelu(self.head3(x))
        x = x + x_res

        # 最終層
        x = self.final_layer(x)
        return x


In [10]:
# model test
model = CustomModel(cfg.cnn)

tmp_image = torch.randn(1, 20, 224, 224)
tmp_numeric_features = torch.randn(1, 8)

tmp_output = model(tmp_image, tmp_numeric_features)

tmp_output.shape


torch.Size([1, 18])

### 学習

In [11]:
DEBUG = False

if DEBUG:
    cfg.cnn.num_epochs = 1
    train = train.head(100)


In [12]:
# 実験結果格納用のディレクトリを作成
japan_tz = pytz.timezone("Asia/Tokyo")
cfg.run_time = datetime.now(japan_tz).strftime("%Y%m%d_%H%M%S")
create_dir(cfg.data.results_path)

# CV用の配列を初期化
oof_predictions = np.zeros((len(train), len(cfg.target_cols)))
models = {}

for fold, (train_idx, valid_idx) in enumerate(gkf.split(train, groups=train["scene"])):
    print(f"Fold {fold + 1}")

    # データセットの作成
    train_dataset = CustomDataset(
        train_target_scaled[train_idx],
        cfg.data.img_root,
        cfg.data.json_root,
        cfg.data.depth_root,
        transform=get_train_transform(),
    )
    valid_dataset = CustomDataset(
        train_target_scaled[valid_idx],
        cfg.data.img_root,
        cfg.data.json_root,
        cfg.data.depth_root,
        transform=get_valid_transform(),
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.cnn.batch_size,
        shuffle=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=cfg.cnn.batch_size,
        shuffle=False,
    )

    # モデル、損失関数、オプティマイザーの初期化
    model = CustomModel(cfg.cnn).to(device)
    # criterion = nn.HuberLoss()
    criterion = nn.L1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.cnn.lr)
    total_steps = len(train_loader) * cfg.cnn.num_epochs
    scheduler = get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=total_steps * 0.1,
        num_training_steps=total_steps,
    )

    best_loss = float("inf")

    # 学習ループ
    for epoch in range(cfg.cnn.num_epochs):
        model.train()
        for images, numeric_features, targets in tqdm(train_loader):
            images = images.to(device)
            numeric_features = numeric_features.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(images, numeric_features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()

        # 検証
        model.eval()
        valid_losses = []
        with torch.no_grad():
            for images, numeric_features, targets in valid_loader:
                images = images.to(device)
                numeric_features = numeric_features.to(device)
                targets = targets.to(device)
                outputs = model(images, numeric_features)
                loss = criterion(outputs, targets)
                valid_losses.append(loss.item())

        valid_loss = np.mean(valid_losses)
        print(f"Epoch {epoch + 1}, Valid Loss: {valid_loss:.4f}")

        # ベストモデルの保存
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), f"{cfg.data.results_path}/model_fold{fold}.pth")

    # ベストモデルでOOF予測
    model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
    model.eval()

    valid_dataset = CustomDataset(
        train_target_scaled[valid_idx],
        cfg.data.img_root,
        cfg.data.json_root,
        cfg.data.depth_root,
        transform=get_valid_transform(),
    )
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

    with torch.no_grad():
        for i, (images, numeric_features, _) in enumerate(valid_loader):
            images = images.to(device)
            numeric_features = numeric_features.to(device)
            outputs = model(images, numeric_features)
            start_idx = i * cfg.cnn.batch_size
            end_idx = start_idx + outputs.shape[0]
            oof_predictions[valid_idx[start_idx:end_idx]] = outputs.cpu().numpy()

# CVスコアの計算（MAEの平均）

# oofのスケールを戻す
oof_predictions = target_scaler.inverse_transform(oof_predictions)
mae_scores = []
for i in range(len(cfg.target_cols)):
    mae = np.mean(np.abs(oof_predictions[:, i] - train[cfg.target_cols[i]].to_numpy()))
    mae_scores.append(mae)

cv_score = np.mean(mae_scores)
print(f"CV Score: {cv_score:.4f}")

# oofを保存
np.save(f"{cfg.data.results_path}/oof_predictions.npy", oof_predictions)


Directory created: ../../results/012/20241125_223138
Fold 1


100%|██████████| 1085/1085 [14:58<00:00,  1.21it/s]


Epoch 1, Valid Loss: 0.3338


100%|██████████| 1085/1085 [09:06<00:00,  1.99it/s]


Epoch 2, Valid Loss: 0.3172


...

Epoch 4, Valid Loss: 0.3098


100%|██████████| 1085/1085 [09:04<00:00,  1.99it/s]


Epoch 5, Valid Loss: 0.3048


100%|██████████| 1085/1085 [08:59<00:00,  2.01it/s]


Epoch 6, Valid Loss: 0.3034
CV Score: 0.2317


### 推論

In [13]:
# testの推論
test_dataset = CustomDataset(
    test, cfg.data.img_root, cfg.data.json_root, cfg.data.depth_root, transform=get_valid_transform(), is_train=False
)
test_loader = DataLoader(test_dataset, batch_size=cfg.cnn.batch_size, shuffle=False)

# fold分の予測値を格納する配列
test_predictions = np.zeros((len(test), len(cfg.target_cols), cfg.n_splits))

# 各foldのモデルで予測
for fold in range(cfg.n_splits):
    print(f"Predicting using fold {fold + 1} model")

    # モデルの読み込み
    model = CustomModel(cfg.cnn).to(device)
    model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
    model.eval()

    fold_predictions = []
    with torch.no_grad():
        for images, numeric_features in tqdm(test_loader):
            images = images.to(device)
            numeric_features = numeric_features.to(device)
            outputs = model(images, numeric_features)
            fold_predictions.append(outputs.cpu().numpy())

    # バッチごとの予測を結合
    fold_predictions = np.concatenate(fold_predictions, axis=0)
    test_predictions[:, :, fold] = fold_predictions

# fold分の予測値の平均を計算
final_predictions = test_predictions.mean(axis=2)

# 推論値のスケールを戻す
final_predictions = target_scaler.inverse_transform(final_predictions)

# submissionファイルの作成
exprs = [pl.Series(final_predictions[:, i]).alias(cfg.target_cols[i]) for i in range(len(cfg.target_cols))]
submission = sample_submission.with_columns(exprs)
submission= submission.drop("index")
submission.write_csv(f"{cfg.data.results_path}/submission.csv")
print("Submission file created!")

# 確認
submission.head()


Predicting using fold 1 model


/tmp/ipykernel_229296/1161565385.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{cfg.data.results_path}/model_fold{fold}.pth"))
100%

Predicting using fold 2 model


100%|██████████| 54/54 [00:18<00:00,  2.87it/s]


Predicting using fold 3 model


...

100%|██████████| 54/54 [00:18<00:00,  2.87it/s]


Predicting using fold 5 model


100%|██████████| 54/54 [00:18<00:00,  2.85it/s]

Submission file created!


x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.495188,-0.039181,-0.000726,3.143584,-0.087143,-0.001501,4.791918,-0.135711,-0.00212,6.442691,-0.176083,-0.000398,8.087469,-0.210921,0.000257,9.663422,-0.242781,0.00303
0.954703,0.311374,-0.003814,1.851908,0.817911,-0.005987,2.590929,1.460414,-0.007047,3.256482,2.170239,-0.01342,3.835674,2.946243,-0.010635,4.192871,3.748976,-0.004953
1.592748,0.016054,0.00036,3.253291,0.030881,0.000265,4.779555,0.040324,-0.000028,6.171623,0.0457,-0.000503,7.418311,0.046988,-0.003007,8.543471,0.051794,-0.005058
0.88163,0.046782,-0.00312,1.753022,0.162773,-0.005505,2.4879,0.366899,-0.008358,3.053479,0.664821,-0.013423,3.520385,1.04099,-0.020913,3.807704,1.474138,-0.032787
0.819554,0.008511,-0.009766,1.403461,0.015606,-0.02418,1.75973,0.01892,-0.041164,1.955308,0.019548,-0.057974,2.017004,0.013897,-0.074355,2.108573,0.011606,-0.088628


In [14]:
# final_predictionsを保存
np.save(f"{cfg.data.results_path}/final_predictions.npy", final_predictions)


In [15]:
final_predictions.shape


(1727, 18)